<a href="https://colab.research.google.com/github/Meet-13/image-Inpainting/blob/main/BTP_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F


bias_act


In [ ]:
!git clone https://github.com/fenglinglwb/MAT.git

Cloning into 'MAT'...
remote: Enumerating objects: 199, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 199 (delta 42), reused 39 (delta 39), pack-reused 148 (from 1)
Receiving objects: 100% (199/199), 18.89 MiB | 15.67 MiB/s, done.
Resolving deltas: 100% (86/86), done.


In [ ]:
import sys
sys.path.append('/content/MAT')
from torch_utils.ops import upfirdn2d
from torch_utils.ops import bias_act


Fully Connected Layer


In [ ]:
import torch
import torch.nn as nn
import numpy as np

# Define Fully Connected Layer with hardcoded bias_act function
class FullyConnectedLayer(nn.Module):
    def __init__(self,
                 in_features,                # Number of input features.
                 out_features,               # Number of output features.
                 bias=True,                  # Apply additive bias before the activation function?
                 activation='linear',        # Activation function: 'relu', 'lrelu', etc.
                 lr_multiplier=1,            # Learning rate multiplier.
                 bias_init=0):               # Initial value for the additive bias.
        super().__init__()
        # Initialize weights and biases
        self.weight = torch.nn.Parameter(torch.randn([out_features, in_features]) / lr_multiplier)
        self.bias = torch.nn.Parameter(torch.full([out_features], np.float32(bias_init))) if bias else None
        self.activation = activation

        # Gain values for weight scaling
        self.weight_gain = lr_multiplier / np.sqrt(in_features)
        self.bias_gain = lr_multiplier

    def forward(self, x):
        # Compute scaled weight
        w = self.weight * self.weight_gain
        b = self.bias
        if b is not None and self.bias_gain != 1:
            b = b * self.bias_gain

        # Matrix multiplication
        x = x.matmul(w.t())

        # Bias addition
        if b is not None:
            x = x + b.reshape([-1 if i == x.ndim-1 else 1 for i in range(x.ndim)])

        # Apply activation
        x = bias_act.bias_act(x, act=self.activation)
        return x

In [ ]:
# Example
# Define input tensor
input_tensor = torch.randn(4, 128)  # Example input

# Initialize Fully Connected Layer
fc_layer = FullyConnectedLayer(in_features=128, out_features=64, activation='relu')

# Apply layer to input tensor
output = fc_layer(input_tensor)

# Print output
print(output)

tensor([[1.7422, 0.0000, 0.0000, 1.6625, 1.9158, 0.1447, 0.1280, 0.0000, 1.5578,
         0.5332, 0.8277, 0.0000, 0.0000, 0.0000, 1.6467, 0.2258, 0.0000, 1.1077,
         0.0000, 0.0000, 1.2268, 1.9813, 0.0000, 0.6802, 3.4428, 0.8143, 0.0000,
         0.0000, 0.4108, 0.0000, 0.0000, 1.0681, 0.7369, 0.0000, 2.2316, 2.1901,
         0.8963, 0.0000, 0.8565, 0.7958, 0.9232, 2.0932, 0.0256, 0.5827, 1.9466,
         0.9894, 2.2214, 0.4001, 0.0000, 0.0586, 0.2943, 0.0000, 2.4943, 0.0000,
         0.0000, 0.0000, 0.0000, 0.2094, 2.3620, 0.0000, 0.0000, 1.5645, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.4738, 0.5696, 0.0714, 0.6723, 0.2987,
         1.6035, 0.0000, 0.0000, 0.0000, 0.5221, 0.0000, 0.0000, 1.5053, 0.0000,
         1.7508, 0.0000, 0.1932, 0.0000, 2.3073, 0.0000, 0.3389, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 2.3375, 1.2715, 0.4967, 0.0000, 1.5017, 0.9657,
         0.0000, 0.3186, 0.0000, 0.0000, 1.3415, 1.8052, 1.7870, 0.1630, 0.0000,
         0

Conv2dlayer

The Conv2dLayer class is designed as a custom convolutional layer in a neural network, similar to what you would find in popular architectures like convolutional neural networks (CNNs)

In [ ]:
# Define Conv2dLayer
class Conv2dLayer(nn.Module):
    def __init__(self,
                 in_channels,                    # Number of input channels.
                 out_channels,                   # Number of output channels.
                 kernel_size,                    # Width and height of the convolution kernel.
                 bias=True,                      # Apply additive bias before the activation function?
                 activation='linear',            # Activation function: 'relu', 'lrelu', etc.
                 up=1,                           # Integer upsampling factor.
                 down=1,                         # Integer downsampling factor.
                 resample_filter=[1, 3, 3, 1],   # Low-pass filter to apply when resampling activations.
                 conv_clamp=None,                # Clamp the output to +-X, None = disable clamping.
                 trainable=True):                # Update the weights of this layer during training?
        super().__init__()

        self.activation = activation
        self.up = up
        self.down = down
        self.conv_clamp = conv_clamp
        self.padding = kernel_size // 2
        self.weight_gain = 1 / np.sqrt(in_channels * (kernel_size ** 2))

        # Initialize weights and biases
        weight = torch.randn([out_channels, in_channels, kernel_size, kernel_size])
        bias = torch.zeros([out_channels]) if bias else None

        if trainable:
            self.weight = torch.nn.Parameter(weight)
            self.bias = torch.nn.Parameter(bias) if bias is not None else None
        else:
            self.register_buffer('weight', weight)
            if bias is not None:
                self.register_buffer('bias', bias)
            else:
                self.bias = None

    def forward(self, x, gain=1):
        # Apply convolution
        w = self.weight * self.weight_gain
        x = F.conv2d(x, w, padding=self.padding)

        # Apply activation, gain, and clamping using bias_act function
        act_gain = gain
        act_clamp = self.conv_clamp * gain if self.conv_clamp is not None else None
        x = bias_act.bias_act(x, self.bias, act=self.activation, gain=act_gain, clamp=act_clamp)

        return x

In [ ]:
# Example
# Define input tensor
input_tensor = torch.randn(4, 3, 32, 32)  # Example input with batch size 4, 3 channels, 32x32 height/width

# Initialize Conv2d Layer
conv_layer = Conv2dLayer(in_channels=3, out_channels=16, kernel_size=3, activation='relu')

# Apply layer to input tensor
output = conv_layer(input_tensor)

# Print output
print(output)


tensor([[[[0.0000e+00, 4.5134e-01, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 1.6759e-01,  ..., 1.2356e+00,
           0.0000e+00, 0.0000e+00],
          [4.9374e-01, 2.2984e-01, 3.7355e-01,  ..., 0.0000e+00,
           7.1948e-01, 6.9212e-01],
          ...,
          [5.8218e-01, 5.9770e-01, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 5.9609e-01],
          [6.7606e-01, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           2.6319e-01, 3.2272e-01],
          [2.7260e-01, 2.4159e-01, 3.2315e-01,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.1750e+00,
           0.0000e+00, 4.4311e-01],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 2.0591e+00,
           0.0000e+00, 0.0000e+00],
          [7.8460e-01, 0.0000e+00, 9.5420e-01,  ..., 0.0000e+00,
           1.0078e+00, 0.0000e+00],
          ...,
          [0.0000e+00, 0.0000e+00, 2.2862e-01,  ..., 1.4348

ModulatedConv2d
The ModulatedConv2d class is a custom convolutional layer that incorporates a modulation mechanism, typically used in advanced generative networks like StyleGAN. The modulation mechanism allows dynamic changes to the convolutional filters based on a style vector, which can help control the output style more finely

In [ ]:
import torch
import torch.nn as nn
import numpy as np

class ModulatedConv2d(nn.Module):
    def __init__(self,
                 in_channels,                   # Number of input channels.
                 out_channels,                  # Number of output channels.
                 kernel_size,                   # Width and height of the convolution kernel.
                 style_dim,                     # Dimension of the style code.
                 demodulate=True,               # Perform demodulation.
                 up=1,                          # Integer upsampling factor.
                 down=1,                        # Integer downsampling factor.
                 resample_filter=[1, 3, 3, 1],  # Low-pass filter to apply when resampling activations.
                 conv_clamp=None,               # Clamp the output to +-X, None = disable clamping.
                 ):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.demodulate = demodulate
        self.up = up
        self.down = down
        self.conv_clamp = conv_clamp
        self.weight_gain = 1 / np.sqrt(in_channels * (kernel_size ** 2))
        self.padding = kernel_size // 2

        # Define weight parameter
        self.weight = nn.Parameter(torch.randn([1, out_channels, in_channels, kernel_size, kernel_size]))

        # Setup resample filter
        self.register_buffer('resample_filter', torch.tensor(resample_filter, dtype=torch.float32))

        # Use the existing FullyConnectedLayer for affine transformation
        self.affine = FullyConnectedLayer(style_dim, in_channels, bias_init=1)

    def forward(self, x, style):
        batch, in_channels, height, width = x.shape

        # Modulation step using the pre-defined affine layer
        style = self.affine(style).view(batch, 1, in_channels, 1, 1)
        weight = self.weight * self.weight_gain * style

        # Demodulation step
        if self.demodulate:
            decoefs = (weight.pow(2).sum(dim=[2, 3, 4]) + 1e-8).rsqrt()
            weight = weight * decoefs.view(batch, self.out_channels, 1, 1, 1)

        # Reshape weight and input for convolution
        weight = weight.view(batch * self.out_channels, in_channels, self.kernel_size, self.kernel_size)
        x = x.view(1, batch * in_channels, height, width)

        # Apply convolution
        out = F.conv2d(x, weight, padding=self.padding, groups=batch)

        # Reshape output
        out = out.view(batch, self.out_channels, *out.shape[2:])

        # Clamp output if needed
        if self.conv_clamp is not None:
            out = out.clamp(-self.conv_clamp, self.conv_clamp)

        return out


In [ ]:
# Example usage
batch_size = 2
in_channels = 3
out_channels = 16
kernel_size = 3
style_dim = 512
height, width = 64, 64

modulated_conv = ModulatedConv2d(in_channels, out_channels, kernel_size, style_dim)
x = torch.randn(batch_size, in_channels, height, width)
style = torch.randn(batch_size, style_dim)

output = modulated_conv(x, style)
print("Output shape:", output.shape)


Output shape: torch.Size([2, 16, 64, 64])


StyleConv
The StyleConv class is designed to perform convolution operations while incorporating style modulation. It combines spatial information with style input, allowing for advanced style transfer applications in generative models.

In [ ]:
class StyleConv(nn.Module):
    def __init__(self,
                 in_channels,                    # Number of input channels.
                 out_channels,                   # Number of output channels.
                 style_dim,                      # Intermediate latent (W) dimensionality.
                 resolution,                     # Resolution of this layer.
                 kernel_size=3,                  # Convolution kernel size.
                 up=1,                           # Integer upsampling factor.
                 use_noise=True,                # Enable noise input?
                 activation='lrelu',            # Activation function: 'relu', 'lrelu', etc.
                 resample_filter=[1, 3, 3, 1],  # Low-pass filter for resampling activations.
                 conv_clamp=None,               # Clamp output of convolution layers to +-X.
                 demodulate=True,                # Perform demodulation.
                 ):
        super().__init__()

        # Initialize the ModulatedConv2d layer
        self.conv = ModulatedConv2d(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=kernel_size,
            style_dim=style_dim,
            demodulate=demodulate,
            up=up,
            resample_filter=resample_filter,
            conv_clamp=conv_clamp
        )

        self.use_noise = use_noise
        self.resolution = resolution
        if use_noise:
            self.register_buffer('noise_const', torch.randn([resolution, resolution]))
            self.noise_strength = nn.Parameter(torch.zeros([]))

        # Bias for the layer
        self.bias = nn.Parameter(torch.zeros([out_channels]))

        # Activation setup
        self.activation = activation
        self.act_gain = 1.0  # You can set this according to your requirements
        self.conv_clamp = conv_clamp

    def forward(self, x, style, noise_mode='random', gain=1):
        # Apply the modulated convolution
        x = self.conv(x, style)

        # Ensure the noise mode is valid
        assert noise_mode in ['random', 'const', 'none']

        # Handle noise if enabled
        if self.use_noise:
            if noise_mode == 'random':
                xh, xw = x.size()[-2:]
                noise = torch.randn([x.shape[0], 1, xh, xw], device=x.device) * self.noise_strength
            elif noise_mode == 'const':
                noise = self.noise_const * self.noise_strength
            else:  # noise_mode == 'none'
                noise = 0

            x = x + noise

        # Apply bias, activation, gain, and clamping using the bias_act function
        out = bias_act.bias_act(x, self.bias, act=self.activation, gain=self.act_gain, clamp=self.conv_clamp)

        return out


In [ ]:
# Example usage
batch_size = 2
in_channels = 3
out_channels = 16
style_dim = 512
resolution = 64
height, width = 64, 64

style_conv = StyleConv(in_channels, out_channels, style_dim, resolution)
x = torch.randn(batch_size, in_channels, height, width)
style = torch.randn(batch_size, style_dim)

output = style_conv(x, style)
print("Output shape:", output.shape)

Output shape: torch.Size([2, 16, 64, 64])


ToRGB
This structure is typical in style-based image generation networks, where the ToRGB layer converts feature maps into RGB images, often incorporating style information for modulating the generated output.

In [ ]:
import torch
import torch.nn as nn

class ToRGB(nn.Module):
    def __init__(self,
                 in_channels,
                 out_channels,
                 style_dim,
                 kernel_size=1,
                 resample_filter=[1,3,3,1],
                 conv_clamp=None,
                 demodulate=False):
        super(ToRGB, self).__init__()

        self.conv = ModulatedConv2d(in_channels=in_channels,
                                    out_channels=out_channels,
                                    kernel_size=kernel_size,
                                    style_dim=style_dim,
                                    demodulate=demodulate,
                                    resample_filter=resample_filter,
                                    conv_clamp=conv_clamp)
        self.bias = nn.Parameter(torch.zeros([out_channels]))
        self.register_buffer('resample_filter', upfirdn2d.setup_filter(resample_filter))
        self.conv_clamp = conv_clamp

    def forward(self, x, style, skip=None):
        x = self.conv(x, style)
        out = bias_act.bias_act(x, self.bias, clamp=self.conv_clamp)

        if skip is not None:
            # If skip and output have different shapes, use interpolation to resize skip to match output size
            if skip.shape != out.shape:
                skip = torch.nn.functional.interpolate(skip, size=out.shape[2:], mode='bilinear', align_corners=False)
            out = out + skip

        return out


In [ ]:
# Define some sample tensors
input_tensor = torch.randn(1, 512, 16, 16).cuda()  # Random input tensor
style_tensor = torch.randn(1, 512).cuda()  # Random style vector
skip_tensor = torch.randn(1, 3, 64, 64).cuda()  # Skip connection (optional)

# Create the ToRGB object
rgb_layer = ToRGB(in_channels=512, out_channels=3, style_dim=512).cuda()

# Run the forward pass
output = rgb_layer(input_tensor, style_tensor, skip_tensor)
print(output.shape)  # Output should be a tensor with shape (1, 3, 16, 16)


Setting up PyTorch plugin "bias_act_plugin"... Failed!


/content/MAT/torch_utils/ops/bias_act.py:50: UserWarning: Failed to build CUDA kernels for bias_act. Falling back to slow reference implementation. Details:

Traceback (most recent call last):
  File "/content/MAT/torch_utils/ops/bias_act.py", line 48, in _init
    _plugin = custom_ops.get_plugin('bias_act_plugin', sources=sources, extra_cuda_cflags=['--use_fast_math'])
  File "/content/MAT/torch_utils/custom_ops.py", line 110, in get_plugin
    torch.utils.cpp_extension.load(name=module_name, verbose=verbose_build, sources=sources, **build_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py", line 1314, in load
    return _jit_compile(
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py", line 1721, in _jit_compile
    _write_ninja_file_and_build_library(
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py", line 1803, in _write_ninja_file_and_build_library
    verify_ninja_availability()
  File "/usr/l

torch.Size([1, 3, 16, 16])


In [ ]:
def get_style_code(a, b):
    return torch.cat([a, b], dim=1)

DecBlockFirst

In [ ]:
class DecBlockFirst(nn.Module):
    def __init__(self, in_channels, out_channels, activation, style_dim, use_noise, demodulate, img_channels):
        super().__init__()
        # Fully connected layer to expand the latent vector into a 4x4 feature map
        self.fc = FullyConnectedLayer(in_features=in_channels*2,
                                      out_features=in_channels*4**2,
                                      activation=activation)

        # Conv layer to adjust the number of channels to out_channels (256)
        self.adjust_channels = nn.Conv2d(in_channels, out_channels, kernel_size=1)

        # Style-based convolution
        self.conv = StyleConv(in_channels=out_channels,
                              out_channels=out_channels,
                              style_dim=style_dim,
                              resolution=4,
                              kernel_size=3,
                              use_noise=use_noise,
                              activation=activation,
                              demodulate=demodulate,
                              )
        # ToRGB layer for converting the feature map into an RGB image
        self.toRGB = ToRGB(in_channels=out_channels,
                           out_channels=img_channels,
                           style_dim=style_dim,
                           kernel_size=1,
                           demodulate=False,
                           )

    def forward(self, x, ws, gs, E_features, noise_mode='random'):
        # Fully connected layer to reshape input latent vector into a 4x4 feature map
        x = self.fc(x).view(x.shape[0], -1, 4, 4)

        # Adjust the number of channels in x to match E_features
        x = self.adjust_channels(x)

        # Add skip connection (feature map from an encoder or earlier layer) to the feature map
        x = x + E_features[2]  # Adds encoder feature map at a corresponding scale

        # Get the style code for the first convolutional layer
        style = get_style_code(ws[:, 0], gs)
        x = self.conv(x, style, noise_mode=noise_mode)

        # Get the style code for the ToRGB layer
        style = get_style_code(ws[:, 1], gs)
        img = self.toRGB(x, style, skip=None)

        return x, img


In [ ]:
# Example
def get_style_code(ws, gs):
    return ws + gs  # For testing, we just sum the style and global style vectors (simplified behavior)

# Create an instance of DecBlockFirst
in_channels = 512    # Example input channel size
out_channels = 256   # Example output channel size
style_dim = 512      # Example style vector dimension
activation = 'relu'  # Example activation function
use_noise = True     # Adding noise to the convolution
demodulate = False   # Not using demodulation in the conv layer
img_channels = 3     # Example image channels (RGB)

dec_block_first = DecBlockFirst(in_channels, out_channels, activation, style_dim, use_noise, demodulate, img_channels)

# Create dummy inputs for testing the forward pass
batch_size = 1  # Single batch
x = torch.randn(batch_size, in_channels * 2)  # Latent vector for fully connected layer

ws = torch.randn(batch_size, 2, style_dim)  # Style vectors for two stages (conv and ToRGB)
gs = torch.randn(batch_size, style_dim)      # Global style vectors

# E_features: List of features from an encoder (assuming 3 feature maps in the encoder)
E_features = [torch.randn(batch_size, out_channels, 4, 4) for _ in range(3)]

# Run the forward pass of DecBlockFirst
x_out, img_out = dec_block_first(x, ws, gs, E_features)

# Print the shape of the output tensors
print("Output feature map shape:", x_out.shape)  # Should be (batch_size, out_channels, 4, 4)
print("Output image shape:", img_out.shape)      # Should be (batch_size, img_channels, 4, 4)


Output feature map shape: torch.Size([1, 256, 4, 4])
Output image shape: torch.Size([1, 3, 4, 4])


In [ ]:
class DecBlockFirstV2(nn.Module):
    def __init__(self, in_channels, out_channels, activation, style_dim, use_noise, demodulate, img_channels):
        super().__init__()
        self.conv0 = Conv2dLayer(in_channels=in_channels,
                                out_channels=in_channels,
                                kernel_size=3,
                                activation=activation,
                                )
        self.conv1 = StyleConv(in_channels=in_channels,
                              out_channels=out_channels,
                              style_dim=style_dim,
                              resolution=4,
                              kernel_size=3,
                              use_noise=use_noise,
                              activation=activation,
                              demodulate=demodulate,
                              )
        self.toRGB = ToRGB(in_channels=out_channels,
                           out_channels=img_channels,
                           style_dim=style_dim,
                           kernel_size=1,
                           demodulate=False,
                           )

    def forward(self, x, ws, gs, E_features, noise_mode='random'):
        x = self.conv0(x)
        x = x + E_features[2]
        style = get_style_code(ws[:, 0], gs)
        x = self.conv1(x, style, noise_mode=noise_mode)
        style = get_style_code(ws[:, 1], gs)
        img = self.toRGB(x, style, skip=None)

        return x, img


In [ ]:
import torch

# Assuming the following classes are defined:
# Conv2dLayer, StyleConv, ToRGB, DecBlockFirstV2

# Create an instance of DecBlockFirstV2
in_channels = 512    # Example input channel size
out_channels = 256   # Example output channel size
style_dim = 512      # Example style vector dimension
activation = 'relu'  # Example activation function
use_noise = True     # Adding noise to the convolution
demodulate = False   # Not using demodulation in the conv layer
img_channels = 3     # Example image channels (RGB)

dec_block_first_v2 = DecBlockFirstV2(in_channels, out_channels, activation, style_dim, use_noise, demodulate, img_channels)

# Create dummy inputs for testing the forward pass
batch_size = 1  # Single batch
x = torch.randn(batch_size, in_channels, 4, 4)  # Input feature map with correct shape

ws = torch.randn(batch_size, 2, style_dim)  # Style vectors for two stages (conv and ToRGB)
gs = torch.randn(batch_size, style_dim)      # Global style vectors

# E_features: List of features from an encoder (assuming 3 feature maps in the encoder)
# Note: Ensure these have the right shape and number of channels
E_features = [torch.randn(batch_size, in_channels, 4, 4) for _ in range(3)]  # Change `out_channels` to `in_channels`

# Run the forward pass of DecBlockFirstV2
x_out, img_out = dec_block_first_v2(x, ws, gs, E_features)

# Print the shape of the output tensors
print("Output feature map shape:", x_out.shape)  # Should be (batch_size, out_channels, 4, 4)
print("Output image shape:", img_out.shape)      # Should be (batch_size, img_channels, 4, 4)


Output feature map shape: torch.Size([1, 256, 4, 4])
Output image shape: torch.Size([1, 3, 4, 4])


In [ ]:
class DecBlock(nn.Module):
    def __init__(self, res, in_channels, out_channels, activation, style_dim, use_noise, demodulate, img_channels):  # res = 2, ..., resolution_log2
        super().__init__()
        self.res = res

        self.conv0 = StyleConv(in_channels=in_channels,
                               out_channels=out_channels,
                               style_dim=style_dim,
                               resolution=2**res,
                               kernel_size=3,
                               up=2,
                               use_noise=use_noise,
                               activation=activation,
                               demodulate=demodulate,
                               )
        self.conv1 = StyleConv(in_channels=out_channels,
                               out_channels=out_channels,
                               style_dim=style_dim,
                               resolution=2**res,
                               kernel_size=3,
                               use_noise=use_noise,
                               activation=activation,
                               demodulate=demodulate,
                               )
        self.toRGB = ToRGB(in_channels=out_channels,
                           out_channels=img_channels,
                           style_dim=style_dim,
                           kernel_size=1,
                           demodulate=False,
                           )

    def forward(self, x, img, ws, gs, E_features, noise_mode='random'):
        style = get_style_code(ws[:, self.res * 2 - 5], gs)
        x = self.conv0(x, style, noise_mode=noise_mode)
        x = x + E_features[self.res]
        style = get_style_code(ws[:, self.res * 2 - 4], gs)
        x = self.conv1(x, style, noise_mode=noise_mode)
        style = get_style_code(ws[:, self.res * 2 - 3], gs)
        img = self.toRGB(x, style, skip=img)

        return x, img

MappingNet

In [ ]:
def normalize_2nd_moment(x, dim=1, eps=1e-8):
    return x * (x.square().mean(dim=dim, keepdim=True) + eps).rsqrt()

In [ ]:
class MappingNet(torch.nn.Module):
    def __init__(self,
        z_dim,                      # Input latent (Z) dimensionality, 0 = no latent.
        c_dim,                      # Conditioning label (C) dimensionality, 0 = no label.
        w_dim,                      # Intermediate latent (W) dimensionality.
        num_ws,                     # Number of intermediate latents to output, None = do not broadcast.
        num_layers      = 8,        # Number of mapping layers.
        embed_features  = None,     # Label embedding dimensionality, None = same as w_dim.
        layer_features  = None,     # Number of intermediate features in the mapping layers, None = same as w_dim.
        activation      = 'lrelu',  # Activation function: 'relu', 'lrelu', etc.
        lr_multiplier   = 0.01,     # Learning rate multiplier for the mapping layers.
        w_avg_beta      = 0.995,    # Decay for tracking the moving average of W during training, None = do not track.
    ):
        super().__init__()
        self.z_dim = z_dim
        self.c_dim = c_dim
        self.w_dim = w_dim
        self.num_ws = num_ws
        self.num_layers = num_layers
        self.w_avg_beta = w_avg_beta

        if embed_features is None:
            embed_features = w_dim
        if c_dim == 0:
            embed_features = 0
        if layer_features is None:
            layer_features = w_dim
        features_list = [z_dim + embed_features] + [layer_features] * (num_layers - 1) + [w_dim]

        if c_dim > 0:
            self.embed = FullyConnectedLayer(c_dim, embed_features)
        for idx in range(num_layers):
            in_features = features_list[idx]
            out_features = features_list[idx + 1]
            layer = FullyConnectedLayer(in_features, out_features, activation=activation, lr_multiplier=lr_multiplier)
            setattr(self, f'fc{idx}', layer)

        if num_ws is not None and w_avg_beta is not None:
            self.register_buffer('w_avg', torch.zeros([w_dim]))

    def forward(self, z, c, truncation_psi=1, truncation_cutoff=None, skip_w_avg_update=False):
        # Embed, normalize, and concat inputs.
        x = None
        with torch.autograd.profiler.record_function('input'):
            if self.z_dim > 0:
                x = normalize_2nd_moment(z.to(torch.float32))
            if self.c_dim > 0:
                c = c.view(-1, 1).expand(-1, self.c_dim)  # or c.unsqueeze(1) depending on your setup
                y = normalize_2nd_moment(self.embed(c.to(torch.float32)))
                x = torch.cat([x, y], dim=1) if x is not None else y

        # Main layers.
        for idx in range(self.num_layers):
            layer = getattr(self, f'fc{idx}')
            x = layer(x)

        # Update moving average of W.
        if self.w_avg_beta is not None and self.training and not skip_w_avg_update:
            with torch.autograd.profiler.record_function('update_w_avg'):
                self.w_avg.copy_(x.detach().mean(dim=0).lerp(self.w_avg, self.w_avg_beta))

        # Broadcast.
        if self.num_ws is not None:
            with torch.autograd.profiler.record_function('broadcast'):
                x = x.unsqueeze(1).repeat([1, self.num_ws, 1])

        # Apply truncation.
        if truncation_psi != 1:
            with torch.autograd.profiler.record_function('truncate'):
                assert self.w_avg_beta is not None
                if self.num_ws is None or truncation_cutoff is None:
                    x = self.w_avg.lerp(x, truncation_psi)
                else:
                    x[:, :truncation_cutoff] = self.w_avg.lerp(x[:, :truncation_cutoff], truncation_psi)

        return x

In [ ]:
import torch

# Example parameters
z_dim = 512              # Latent vector dimension
c_dim = 10               # Conditioning dimension (e.g., class labels)
w_dim = 512              # Intermediate latent space dimension
num_ws = 8               # Number of output latents
num_layers = 8           # Number of layers in the mapping network

# Create the mapping network
mapping_net = MappingNet(z_dim, c_dim, w_dim, num_ws, num_layers)

# Example inputs
batch_size = 4
z = torch.randn(batch_size, z_dim)  # Random latent vectors
c = torch.randint(0, c_dim, (batch_size,))  # Random conditioning labels

# Forward pass
output = mapping_net(z, c)

# Print output shape
print("Output shape:", output.shape)  # Should be (batch_size, num_ws, w_dim)


Output shape: torch.Size([4, 8, 512])


DisFromRGB

In [ ]:
class DisFromRGB(nn.Module):
    def __init__(self, in_channels, out_channels, activation):  # res = 2, ..., resolution_log2
        super().__init__()
        self.conv = Conv2dLayer(in_channels=in_channels,
                                out_channels=out_channels,
                                kernel_size=1,
                                activation=activation,
                                )

    def forward(self, x):
        return self.conv(x)

DisBlock

In [ ]:
class DisBlock(nn.Module):
    def __init__(self, in_channels, out_channels, activation):  # res = 2, ..., resolution_log2
        super().__init__()
        self.conv0 = Conv2dLayer(in_channels=in_channels,
                                 out_channels=in_channels,
                                 kernel_size=3,
                                 activation=activation,
                                 )
        self.conv1 = Conv2dLayer(in_channels=in_channels,
                                 out_channels=out_channels,
                                 kernel_size=3,
                                 down=2,
                                 activation=activation,
                                 )
        self.skip = Conv2dLayer(in_channels=in_channels,
                                out_channels=out_channels,
                                kernel_size=1,
                                down=2,
                                bias=False,
                             )

    def forward(self, x):
        skip = self.skip(x, gain=np.sqrt(0.5))
        x = self.conv0(x)
        x = self.conv1(x, gain=np.sqrt(0.5))
        out = skip + x

        return out

In [ ]:
in_channels = 512
out_channels = 256
activation = 'relu'

dis_block = DisBlock(in_channels, out_channels, activation)

# Create a dummy input tensor
batch_size = 2
height = 8
width = 8
x = torch.randn(batch_size, in_channels, height, width)

# Forward pass
output = dis_block(x)

# Print output shape
print(x.shape)
print("Output shape:", output.shape)  # Should be (batch_size, out_channels, height // 2, width // 2)

torch.Size([2, 512, 8, 8])
Output shape: torch.Size([2, 256, 8, 8])


MiniBatchStdLayer

In [ ]:
class MinibatchStdLayer(torch.nn.Module):
    def __init__(self, group_size, num_channels=1):
        super().__init__()
        self.group_size = group_size
        self.num_channels = num_channels

    def forward(self, x):
        N, C, H, W = x.shape

        # Calculate the effective group size
        G = min(self.group_size, N) if self.group_size is not None else N
        F = self.num_channels
        c = C // F

        # Reshape the input tensor into groups
        y = x.reshape(G, -1, F, c, H, W)  # [GnFcHW]
        y = y - y.mean(dim=0)  # Subtract mean over group
        y = y.square().mean(dim=0)  # Calculate variance over group
        y = (y + 1e-8).sqrt()  # Calculate stddev over group
        y = y.mean(dim=[2, 3, 4])  # Average over channels and pixels
        y = y.reshape(-1, F, 1, 1)  # Add missing dimensions
        y = y.repeat(G, 1, H, W)  # Replicate over group and pixels

        # Concatenate the stddev to the input
        x = torch.cat([x, y], dim=1)  # [NCHW]
        return x


In [ ]:
# Example usage:
batch_size = 4  # Number of samples in a batch
channels = 3    # Number of input channels (e.g., RGB)
height = 8      # Height of the feature maps
width = 8       # Width of the feature maps

# Create a random input tensor
x = torch.randn(batch_size, channels, height, width)

# Create an instance of MinibatchStdLayer
minibatch_std_layer = MinibatchStdLayer(group_size=2, num_channels=1)

# Forward pass
output = minibatch_std_layer(x)

# Print the output shape
print("Output shape:", output.shape)

Output shape: torch.Size([4, 4, 8, 8])


Discriminator

In [ ]:
class Discriminator(torch.nn.Module):
    def __init__(self,
                 c_dim,                        # Conditioning label (C) dimensionality.
                 img_resolution,               # Input resolution.
                 img_channels,                 # Number of input color channels.
                 channel_base       = 32768,    # Overall multiplier for the number of channels.
                 channel_max        = 512,      # Maximum number of channels in any layer.
                 channel_decay      = 1,
                 cmap_dim           = None,     # Dimensionality of mapped conditioning label, None = default.
                 activation         = 'lrelu',
                 mbstd_group_size   = 4,        # Group size for the minibatch standard deviation layer, None = entire minibatch.
                 mbstd_num_channels = 1,        # Number of features for the minibatch standard deviation layer, 0 = disable.
                 ):
        super().__init__()
        self.c_dim = c_dim
        self.img_resolution = img_resolution
        self.img_channels = img_channels

        resolution_log2 = int(np.log2(img_resolution))
        assert img_resolution == 2 ** resolution_log2 and img_resolution >= 4
        self.resolution_log2 = resolution_log2

        def nf(stage):
            return np.clip(int(channel_base / 2 ** (stage * channel_decay)), 1, channel_max)

        if cmap_dim == None:
            cmap_dim = nf(2)
        if c_dim == 0:
            cmap_dim = 0
        self.cmap_dim = cmap_dim

        if c_dim > 0:
            self.mapping = MappingNet(z_dim=0, c_dim=c_dim, w_dim=cmap_dim, num_ws=None, w_avg_beta=None)

        Dis = [DisFromRGB(img_channels+1, nf(resolution_log2), activation)]
        for res in range(resolution_log2, 2, -1):
            Dis.append(DisBlock(nf(res), nf(res-1), activation))

        if mbstd_num_channels > 0:
            Dis.append(MinibatchStdLayer(group_size=mbstd_group_size, num_channels=mbstd_num_channels))
        Dis.append(Conv2dLayer(nf(2) + mbstd_num_channels, nf(2), kernel_size=3, activation=activation))
        self.Dis = nn.Sequential(*Dis)

        self.fc0 = FullyConnectedLayer(nf(2)*4**2, nf(2), activation=activation)
        self.fc1 = FullyConnectedLayer(nf(2), 1 if cmap_dim == 0 else cmap_dim)

    def forward(self, images_in, masks_in, c):
        x = torch.cat([masks_in - 0.5, images_in], dim=1)
        x = self.Dis(x)
        x = self.fc1(self.fc0(x.flatten(start_dim=1)))

        if self.c_dim > 0:
            cmap = self.mapping(None, c)

        if self.cmap_dim > 0:
            x = (x * cmap).sum(dim=1, keepdim=True) * (1 / np.sqrt(self.cmap_dim))

        return x

In [ ]:
def nf(stage, channel_base=32768, channel_decay=1.0, channel_max=512):
    NF = {512: 64, 256: 128, 128: 256, 64: 512, 32: 512, 16: 512, 8: 512, 4: 512}
    return NF[2 ** stage]

MLP layer

In [ ]:
class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = FullyConnectedLayer(in_features=in_features, out_features=hidden_features, activation='lrelu')
        self.fc2 = FullyConnectedLayer(in_features=hidden_features, out_features=out_features)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [ ]:
# Example
# Input feature size of 128, hidden size of 64, output size of 32
# Using a dropout of 0.5 and GELU activation
mlp_model = Mlp(in_features=128, hidden_features=64, out_features=32, act_layer=torch.nn.GELU, drop=0.5)

# Create a random tensor to simulate a batch of inputs
# Assuming a batch size of 10 with 128 features per input
input_tensor = torch.randn(10, 128)

# Perform a forward pass through the model
output_tensor = mlp_model(input_tensor)

# Print output shape and values for verification
print("Output shape:", output_tensor.shape)  # Expected shape: (10, 32)
print("Output values:", output_tensor)


Output shape: torch.Size([10, 32])
Output values: tensor([[ 1.9070, -1.8160,  0.3538,  0.5282,  1.0192,  0.3757,  0.2851, -0.6292,
         -1.3916,  1.7813,  0.6381,  0.4187, -1.1728,  1.0477, -0.4216,  1.4196,
          0.0097,  0.5228, -0.9435,  1.2384,  0.3905, -0.9183,  2.0544,  0.1904,
         -0.8917,  0.0949,  2.1079,  0.1109, -0.6154,  0.5166, -0.7709,  0.0137],
        [-0.9681, -1.1288,  0.3563,  0.6750, -0.0262, -0.8574, -0.1819, -1.4985,
          0.2216,  1.0489, -0.9350, -1.0763, -0.8498, -1.4554,  0.3384,  0.6307,
         -0.3702, -1.3922, -0.1444, -2.0599,  0.7344,  1.4922, -0.7892,  1.1897,
         -0.6544,  1.5134, -0.6282, -1.5124,  1.0093, -0.0036,  0.5671,  0.2046],
        [ 0.8011, -1.3428,  0.3524,  1.5907,  0.9532,  1.9668, -0.5615, -0.6193,
         -0.8348, -0.2388, -0.7735,  0.2250, -2.3748, -1.3422, -1.3789, -0.5028,
          1.4112,  0.2426,  0.4453, -0.6862, -1.2582, -1.6452,  1.4368,  1.6692,
          0.4263, -0.2722,  2.1582,  1.4501, -0.9480, -0.

window partition

In [ ]:
def window_partition(x, window_size):
    """
    Args:
        x: (B, H, W, C)
        window_size (int): window size
    Returns:
        windows: (num_windows*B, window_size, window_size, C)
    """
    B, H, W, C = x.shape
    x = x.view(B, H // window_size, window_size, W // window_size, window_size, C)
    windows = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(-1, window_size, window_size, C)
    return windows

In [ ]:
# Example
import torch

# Example input tensor of shape (Batch, Height, Width, Channels)
B, H, W, C = 2, 8, 8, 3  # 1 batch, 8x8 grid, 3 channels
x = torch.arange(B * H * W * C).view(B, H, W, C).float()

window_size = 4  # Specify window size

# Use the window_partition function
windows = window_partition(x, window_size)

print("Input tensor shape:", x.shape)
print("Partitioned windows shape:", windows.shape)


Input tensor shape: torch.Size([2, 8, 8, 3])
Partitioned windows shape: torch.Size([8, 4, 4, 3])


Window reverse

In [ ]:
def window_reverse(windows, window_size, H, W):
    """
    Args:
        windows: (num_windows*B, window_size, window_size, C)
        window_size (int): Window size
        H (int): Height of image
        W (int): Width of image
    Returns:
        x: (B, H, W, C)
    """
    B = int(windows.shape[0] / (H * W / window_size / window_size))
    x = windows.view(B, H // window_size, W // window_size, window_size, window_size, -1)
    x = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(B, H, W, -1)
    return x

In [ ]:
# Example
import torch

# Set parameters for the test
B, H, W, C = 2, 8, 8, 3    # Batch size, image height, width, and channels
window_size = 4            # Window size

# Create a sample tensor and partition it into windows
input_tensor = torch.randn(B, H, W, C)
windows = window_partition(input_tensor, window_size)

# Reconstruct the image using window_reverse
output_tensor = window_reverse(windows, window_size, H, W)

# Print output shapes and check if they match the original input
print("Original shape:", input_tensor.shape)         # Expected shape: (B, H, W, C)
print("Reconstructed shape:", output_tensor.shape)    # Expected shape: (B, H, W, C)

# Verify if reconstruction matches the original input
if torch.allclose(input_tensor, output_tensor):
    print("The reconstruction is correct!")
else:
    print("The reconstruction differs from the original input.")


Original shape: torch.Size([2, 8, 8, 3])
Reconstructed shape: torch.Size([2, 8, 8, 3])
The reconstruction is correct!


Conv2dLayerPartial

In [ ]:
class Conv2dLayerPartial(nn.Module):
    def __init__(self,
                 in_channels,                    # Number of input channels.
                 out_channels,                   # Number of output channels.
                 kernel_size,                    # Width and height of the convolution kernel.
                 bias            = True,         # Apply additive bias before the activation function?
                 activation      = 'linear',     # Activation function: 'relu', 'lrelu', etc.
                 up              = 1,            # Integer upsampling factor.
                 down            = 1,            # Integer downsampling factor.
                 resample_filter = [1,3,3,1],    # Low-pass filter to apply when resampling activations.
                 conv_clamp      = None,         # Clamp the output to +-X, None = disable clamping.
                 trainable       = True,         # Update the weights of this layer during training?
                 ):
        super().__init__()
        self.conv = Conv2dLayer(in_channels, out_channels, kernel_size, bias, activation, up, down, resample_filter,
                                conv_clamp, trainable)

        self.weight_maskUpdater = torch.ones(1, 1, kernel_size, kernel_size)
        self.slide_winsize = kernel_size ** 2
        self.stride = down
        self.padding = kernel_size // 2 if kernel_size % 2 == 1 else 0

    def forward(self, x, mask=None):
        if mask is not None:
            with torch.no_grad():
                if self.weight_maskUpdater.type() != x.type():
                    self.weight_maskUpdater = self.weight_maskUpdater.to(x)
                update_mask = F.conv2d(mask, self.weight_maskUpdater, bias=None, stride=self.stride, padding=self.padding)
                mask_ratio = self.slide_winsize / (update_mask + 1e-8)
                update_mask = torch.clamp(update_mask, 0, 1)  # 0 or 1
                mask_ratio = torch.mul(mask_ratio, update_mask)
            x = self.conv(x)
            x = torch.mul(x, mask_ratio)
            return x, update_mask
        else:
            x = self.conv(x)
            return x, None

In [ ]:
# Example
import torch
import torch.nn.functional as F

# Initialize Conv2dLayerPartial layer
conv_layer = Conv2dLayerPartial(in_channels=3, out_channels=3, kernel_size=3, activation='lrelu')

# Create an input tensor and a mask
input_tensor = torch.randn(1, 3, 8, 8)      # Batch size = 1, Channels = 3, Height = 8, Width = 8
mask = torch.ones(1, 1, 8, 8)               # Mask tensor, with same spatial dimensions as input

# Add a "hole" in the mask for testing
mask[:, :, 3:5, 3:5] = 0  # Set part of the mask to zero

# Forward pass with the mask
output, updated_mask = conv_layer(input_tensor, mask)

# Print output and mask shapes
print("Output shape:", output.shape)               # Expected: (1, 3, 8, 8)
print("Updated mask shape:", updated_mask.shape)    # Expected: (1, 1, 8, 8)
print("Updated mask:\n", updated_mask[0, 0].cpu())  # Display the updated mask for verification

Output shape: torch.Size([1, 3, 8, 8])
Updated mask shape: torch.Size([1, 1, 8, 8])
Updated mask:
 tensor([[1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])




 Window based multi-head self attention (W-MSA) module


In [ ]:
class WindowAttention(nn.Module):
    r""" Window based multi-head self attention (W-MSA) module with relative position bias.
    It supports both of shifted and non-shifted window.
    Args:
        dim (int): Number of input channels.
        window_size (tuple[int]): The height and width of the window.
        num_heads (int): Number of attention heads.
        qkv_bias (bool, optional):  If True, add a learnable bias to query, key, value. Default: True
        qk_scale (float | None, optional): Override default qk scale of head_dim ** -0.5 if set
        attn_drop (float, optional): Dropout ratio of attention weight. Default: 0.0
        proj_drop (float, optional): Dropout ratio of output. Default: 0.0
    """

    def __init__(self, dim, window_size, num_heads, down_ratio=1, qkv_bias=True, qk_scale=None, attn_drop=0., proj_drop=0.):

        super().__init__()
        self.dim = dim
        self.window_size = window_size  # Wh, Ww
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5

        self.q = FullyConnectedLayer(in_features=dim, out_features=dim)
        self.k = FullyConnectedLayer(in_features=dim, out_features=dim)
        self.v = FullyConnectedLayer(in_features=dim, out_features=dim)
        self.proj = FullyConnectedLayer(in_features=dim, out_features=dim)

        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x, mask_windows=None, mask=None):
        """
        Args:
            x: input features with shape of (num_windows*B, N, C)
            mask: (0/-inf) mask with shape of (num_windows, Wh*Ww, Wh*Ww) or None
        """
        B_, N, C = x.shape
        norm_x = F.normalize(x, p=2.0, dim=-1)
        q = self.q(norm_x).reshape(B_, N, self.num_heads, C // self.num_heads).permute(0, 2, 1, 3)
        k = self.k(norm_x).view(B_, -1, self.num_heads, C // self.num_heads).permute(0, 2, 3, 1)
        v = self.v(x).view(B_, -1, self.num_heads, C // self.num_heads).permute(0, 2, 1, 3)

        attn = (q @ k) * self.scale

        if mask is not None:
            nW = mask.shape[0]
            attn = attn.view(B_ // nW, nW, self.num_heads, N, N) + mask.unsqueeze(1).unsqueeze(0)
            attn = attn.view(-1, self.num_heads, N, N)

        if mask_windows is not None:
            attn_mask_windows = mask_windows.squeeze(-1).unsqueeze(1).unsqueeze(1)
            attn = attn + attn_mask_windows.masked_fill(attn_mask_windows == 0, float(-100.0)).masked_fill(
                attn_mask_windows == 1, float(0.0))
            with torch.no_grad():
                mask_windows = torch.clamp(torch.sum(mask_windows, dim=1, keepdim=True), 0, 1).repeat(1, N, 1)

        attn = self.softmax(attn)

        x = (attn @ v).transpose(1, 2).reshape(B_, N, C)
        x = self.proj(x)
        return x, mask_windows


In [ ]:
import torch

# Define parameters
dim = 64               # Input dimension
window_size = (4, 4)   # Window height and width
num_heads = 8          # Number of attention heads
batch_size = 2         # Batch size
num_windows = 16       # Number of windows
N = window_size[0] * window_size[1]  # Number of tokens per window (assuming square windows)

# Initialize WindowAttention module
window_attention = WindowAttention(dim=dim, window_size=window_size, num_heads=num_heads)

# Create dummy input tensor
x = torch.randn(num_windows * batch_size, N, dim)  # Shape: (num_windows * batch_size, N, dim)

# Forward pass through the module
output, mask_windows = window_attention(x)

# Print output shapes
print("Output shape:", output.shape)          # Expected: (num_windows * batch_size, N, dim)
if mask_windows is not None:
    print("Mask windows shape:", mask_windows.shape)


Output shape: torch.Size([32, 16, 64])


Swin Transformer Block

In [ ]:
class SwinTransformerBlock(nn.Module):
    r""" Swin Transformer Block.
    Args:
        dim (int): Number of input channels.
        input_resolution (tuple[int]): Input resulotion.
        num_heads (int): Number of attention heads.
        window_size (int): Window size.
        shift_size (int): Shift size for SW-MSA.
        mlp_ratio (float): Ratio of mlp hidden dim to embedding dim.
        qkv_bias (bool, optional): If True, add a learnable bias to query, key, value. Default: True
        qk_scale (float | None, optional): Override default qk scale of head_dim ** -0.5 if set.
        drop (float, optional): Dropout rate. Default: 0.0
        attn_drop (float, optional): Attention dropout rate. Default: 0.0
        drop_path (float, optional): Stochastic depth rate. Default: 0.0
        act_layer (nn.Module, optional): Activation layer. Default: nn.GELU
        norm_layer (nn.Module, optional): Normalization layer.  Default: nn.LayerNorm
    """

    def __init__(self, dim, input_resolution, num_heads, down_ratio=1, window_size=7, shift_size=0,
                 mlp_ratio=4., qkv_bias=True, qk_scale=None, drop=0., attn_drop=0., drop_path=0.,
                 act_layer=nn.GELU, norm_layer=nn.LayerNorm):
        super().__init__()
        self.dim = dim
        self.input_resolution = input_resolution
        self.num_heads = num_heads
        self.window_size = window_size
        self.shift_size = shift_size
        self.mlp_ratio = mlp_ratio
        if min(self.input_resolution) <= self.window_size:
            # if window size is larger than input resolution, we don't partition windows
            self.shift_size = 0
            self.window_size = min(self.input_resolution)
        assert 0 <= self.shift_size < self.window_size, "shift_size must in 0-window_size"

        if self.shift_size > 0:
            down_ratio = 1
        self.attn = WindowAttention(dim, window_size=to_2tuple(self.window_size), num_heads=num_heads,
                                    down_ratio=down_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop,
                                    proj_drop=drop)

        self.fuse = FullyConnectedLayer(in_features=dim * 2, out_features=dim, activation='lrelu')

        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

        if self.shift_size > 0:
            attn_mask = self.calculate_mask(self.input_resolution)
        else:
            attn_mask = None

        self.register_buffer("attn_mask", attn_mask)

    def calculate_mask(self, x_size):
        # calculate attention mask for SW-MSA
        H, W = x_size
        img_mask = torch.zeros((1, H, W, 1))  # 1 H W 1
        h_slices = (slice(0, -self.window_size),
                    slice(-self.window_size, -self.shift_size),
                    slice(-self.shift_size, None))
        w_slices = (slice(0, -self.window_size),
                    slice(-self.window_size, -self.shift_size),
                    slice(-self.shift_size, None))
        cnt = 0
        for h in h_slices:
            for w in w_slices:
                img_mask[:, h, w, :] = cnt
                cnt += 1

        mask_windows = window_partition(img_mask, self.window_size)  # nW, window_size, window_size, 1
        mask_windows = mask_windows.view(-1, self.window_size * self.window_size)
        attn_mask = mask_windows.unsqueeze(1) - mask_windows.unsqueeze(2)
        attn_mask = attn_mask.masked_fill(attn_mask != 0, float(-100.0)).masked_fill(attn_mask == 0, float(0.0))

        return attn_mask

    def forward(self, x, x_size, mask=None):
        # H, W = self.input_resolution
        H, W = x_size
        B, L, C = x.shape
        assert L == H * W, "input feature has wrong size"

        shortcut = x
        x = x.view(B, H, W, C)
        if mask is not None:
            mask = mask.view(B, H, W, 1)

        # cyclic shift
        if self.shift_size > 0:
            shifted_x = torch.roll(x, shifts=(-self.shift_size, -self.shift_size), dims=(1, 2))
            if mask is not None:
                shifted_mask = torch.roll(mask, shifts=(-self.shift_size, -self.shift_size), dims=(1, 2))
        else:
            shifted_x = x
            if mask is not None:
                shifted_mask = mask

        # partition windows
        x_windows = window_partition(shifted_x, self.window_size)  # nW*B, window_size, window_size, C
        x_windows = x_windows.view(-1, self.window_size * self.window_size, C)  # nW*B, window_size*window_size, C
        if mask is not None:
            mask_windows = window_partition(shifted_mask, self.window_size)
            mask_windows = mask_windows.view(-1, self.window_size * self.window_size, 1)
        else:
            mask_windows = None

        # W-MSA/SW-MSA (to be compatible for testing on images whose shapes are the multiple of window size
        if self.input_resolution == x_size:
            attn_windows, mask_windows = self.attn(x_windows, mask_windows, mask=self.attn_mask)  # nW*B, window_size*window_size, C
        else:
            attn_windows, mask_windows = self.attn(x_windows, mask_windows, mask=self.calculate_mask(x_size).to(x.device))  # nW*B, window_size*window_size, C

        # merge windows
        attn_windows = attn_windows.view(-1, self.window_size, self.window_size, C)
        shifted_x = window_reverse(attn_windows, self.window_size, H, W)  # B H' W' C
        if mask is not None:
            mask_windows = mask_windows.view(-1, self.window_size, self.window_size, 1)
            shifted_mask = window_reverse(mask_windows, self.window_size, H, W)

        # reverse cyclic shift
        if self.shift_size > 0:
            x = torch.roll(shifted_x, shifts=(self.shift_size, self.shift_size), dims=(1, 2))
            if mask is not None:
                mask = torch.roll(shifted_mask, shifts=(self.shift_size, self.shift_size), dims=(1, 2))
        else:
            x = shifted_x
            if mask is not None:
                mask = shifted_mask
        x = x.view(B, H * W, C)
        if mask is not None:
            mask = mask.view(B, H * W, 1)

        # FFN
        x = self.fuse(torch.cat([shortcut, x], dim=-1))
        x = self.mlp(x)

        return x, mask


In [ ]:
import torch

# Parameters for the SwinTransformerBlock
dim = 96                    # Number of input channels
input_resolution = (28, 28) # Resolution of the input feature map (e.g., 28x28)
num_heads = 4               # Number of attention heads
window_size = 7             # Window size for W-MSA
shift_size = 3              # Shift size for SW-MSA
mlp_ratio = 4.0             # Ratio of MLP hidden dim to embedding dim

# Initialize the SwinTransformerBlock
swin_block = SwinTransformerBlock(
    dim=dim,
    input_resolution=input_resolution,
    num_heads=num_heads,
    window_size=window_size,
    shift_size=shift_size,
    mlp_ratio=mlp_ratio
)

# Dummy input tensor: (batch_size, H*W, dim)
batch_size = 2
H, W = input_resolution
L = H * W
x = torch.randn(batch_size, L, dim)

# Dummy mask (optional): Same shape as x with only 1 channel
mask = torch.ones(batch_size, L, 1)

# Forward pass
output, output_mask = swin_block(x, x_size=input_resolution, mask=mask)

# Print output shapes
print("Output shape:", output.shape)        # Expected: (batch_size, H*W, dim)
if output_mask is not None:
    print("Output mask shape:", output_mask.shape)


Output shape: torch.Size([2, 784, 96])
Output mask shape: torch.Size([2, 784, 1])


In [ ]:
from timm.models.layers import DropPath, to_2tuple, trunc_normal_

/usr/local/lib/python3.10/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


PatchMerging module

In [ ]:
class PatchMerging(nn.Module):
    def __init__(self, in_channels, out_channels, down=2):
        super().__init__()
        self.conv = Conv2dLayerPartial(in_channels=in_channels,
                                       out_channels=out_channels,
                                       kernel_size=3,
                                       activation='lrelu',
                                       down=down,
                                       )
        self.down = down

    def forward(self, x, x_size, mask=None):
        x = token2feature(x, x_size)
        if mask is not None:
            mask = token2feature(mask, x_size)
        x, mask = self.conv(x, mask)
        if self.down != 1:
            ratio = 1 / self.down
            x_size = (int(x_size[0] * ratio), int(x_size[1] * ratio))
        x = feature2token(x)
        if mask is not None:
            mask = feature2token(mask)
        return x, x_size, mask


In [ ]:
def token2feature(x, x_size):
    B, N, C = x.shape
    h, w = x_size
    x = x.permute(0, 2, 1).reshape(B, C, h, w)
    return x


def feature2token(x):
    B, C, H, W = x.shape
    x = x.view(B, C, -1).transpose(1, 2)
    return x


Patchupsampling

In [ ]:
class PatchUpsampling(nn.Module):
    def __init__(self, in_channels, out_channels, up=2):
        super().__init__()
        self.conv = Conv2dLayerPartial(in_channels=in_channels,
                                       out_channels=out_channels,
                                       kernel_size=3,
                                       activation='lrelu',
                                       up=up,
                                       )
        self.up = up

    def forward(self, x, x_size, mask=None):
        x = token2feature(x, x_size)
        if mask is not None:
            mask = token2feature(mask, x_size)
        x, mask = self.conv(x, mask)
        if self.up != 1:
            x_size = (int(x_size[0] * self.up), int(x_size[1] * self.up))
        x = feature2token(x)
        if mask is not None:
            mask = feature2token(mask)
        return x, x_size, mask


In [ ]:
batch_size = 2
num_patches = 784  # Assuming initial patch resolution is 28x28
in_channels = 96
out_channels = 192
input_resolution = (28, 28)
up_factor = 2

# Create example input data
x = torch.randn(batch_size, num_patches, in_channels)
mask = torch.ones(batch_size, num_patches, 1)  # Example mask

# Initialize PatchUpsampling
patch_upsampling = PatchUpsampling(in_channels=in_channels, out_channels=out_channels, up=up_factor)

# Run forward pass
output, output_size, output_mask = patch_upsampling(x, input_resolution, mask)

# Print output shapes
print("Output shape:", output.shape)         # Expected: [batch_size, new_num_patches, out_channels]
print("Output resolution:", output_size)     # Expected: (56, 56)
print("Output mask shape:", output_mask.shape if output_mask is not None else None)

Output shape: torch.Size([2, 784, 192])
Output resolution: (56, 56)
Output mask shape: torch.Size([2, 784, 1])


Basic Layer

In [ ]:
class BasicLayer(nn.Module):
    """ A basic Swin Transformer layer for one stage.
    Args:
        dim (int): Number of input channels.
        input_resolution (tuple[int]): Input resolution.
        depth (int): Number of blocks.
        num_heads (int): Number of attention heads.
        window_size (int): Local window size.
        mlp_ratio (float): Ratio of mlp hidden dim to embedding dim.
        qkv_bias (bool, optional): If True, add a learnable bias to query, key, value. Default: True
        qk_scale (float | None, optional): Override default qk scale of head_dim ** -0.5 if set.
        drop (float, optional): Dropout rate. Default: 0.0
        attn_drop (float, optional): Attention dropout rate. Default: 0.0
        drop_path (float | tuple[float], optional): Stochastic depth rate. Default: 0.0
        norm_layer (nn.Module, optional): Normalization layer. Default: nn.LayerNorm
        downsample (nn.Module | None, optional): Downsample layer at the end of the layer. Default: None
        use_checkpoint (bool): Whether to use checkpointing to save memory. Default: False.
    """

    def __init__(self, dim, input_resolution, depth, num_heads, window_size, down_ratio=1,
                 mlp_ratio=2., qkv_bias=True, qk_scale=None, drop=0., attn_drop=0.,
                 drop_path=0., norm_layer=nn.LayerNorm, downsample=None, use_checkpoint=False):

        super().__init__()
        self.dim = dim
        self.input_resolution = input_resolution
        self.depth = depth
        self.use_checkpoint = use_checkpoint

        # patch merging layer
        if downsample is not None:
            # self.downsample = downsample(input_resolution, dim=dim, norm_layer=norm_layer)
            self.downsample = downsample
        else:
            self.downsample = None

        # build blocks
        self.blocks = nn.ModuleList([
            SwinTransformerBlock(dim=dim, input_resolution=input_resolution,
                                 num_heads=num_heads, down_ratio=down_ratio, window_size=window_size,
                                 shift_size=0 if (i % 2 == 0) else window_size // 2,
                                 mlp_ratio=mlp_ratio,
                                 qkv_bias=qkv_bias, qk_scale=qk_scale,
                                 drop=drop, attn_drop=attn_drop,
                                 drop_path=drop_path[i] if isinstance(drop_path, list) else drop_path,
                                 norm_layer=norm_layer)
            for i in range(depth)])

        self.conv = Conv2dLayerPartial(in_channels=dim, out_channels=dim, kernel_size=3, activation='lrelu')

    def forward(self, x, x_size, mask=None):
        if self.downsample is not None:
            x, x_size, mask = self.downsample(x, x_size, mask)
        identity = x
        for blk in self.blocks:
            if self.use_checkpoint:
                x, mask = checkpoint.checkpoint(blk, x, x_size, mask)
            else:
                x, mask = blk(x, x_size, mask)
        if mask is not None:
            mask = token2feature(mask, x_size)
        x, mask = self.conv(token2feature(x, x_size), mask)
        x = feature2token(x) + identity
        if mask is not None:
            mask = feature2token(mask)
        return x, x_size, mask


In [ ]:
class ToToken(nn.Module):
    def __init__(self, in_channels=3, dim=128, kernel_size=5, stride=1):
        super().__init__()

        self.proj = Conv2dLayerPartial(in_channels=in_channels, out_channels=dim, kernel_size=kernel_size, activation='lrelu')

    def forward(self, x, mask):
        x, mask = self.proj(x, mask)

        return x, mask

In [ ]:
class EncFromRGB(nn.Module):
    def __init__(self, in_channels, out_channels, activation):  # res = 2, ..., resolution_log2
        super().__init__()
        self.conv0 = Conv2dLayer(in_channels=in_channels,
                                out_channels=out_channels,
                                kernel_size=1,
                                activation=activation,
                                )
        self.conv1 = Conv2dLayer(in_channels=out_channels,
                                out_channels=out_channels,
                                kernel_size=3,
                                activation=activation,
                                )

    def forward(self, x):
        x = self.conv0(x)
        x = self.conv1(x)

        return x

In [ ]:
class ConvBlockDown(nn.Module):
    def __init__(self, in_channels, out_channels, activation):  # res = 2, ..., resolution_log
        super().__init__()

        self.conv0 = Conv2dLayer(in_channels=in_channels,
                                 out_channels=out_channels,
                                 kernel_size=3,
                                 activation=activation,
                                 down=2,
                                 )
        self.conv1 = Conv2dLayer(in_channels=out_channels,
                                 out_channels=out_channels,
                                 kernel_size=3,
                                 activation=activation,
                                 )

    def forward(self, x):
        x = self.conv0(x)
        x = self.conv1(x)

        return x



In [ ]:
class Encoder(nn.Module):
    def __init__(self, res_log2, img_channels, activation, patch_size=5, channels=16, drop_path_rate=0.1):
        super().__init__()

        self.resolution = []

        for idx, i in enumerate(range(res_log2, 3, -1)):  # from input size to 16x16
            res = 2 ** i
            self.resolution.append(res)
            if i == res_log2:
                block = EncFromRGB(img_channels * 2 + 1, nf(i), activation)
            else:
                block = ConvBlockDown(nf(i+1), nf(i), activation)
            setattr(self, 'EncConv_Block_%dx%d' % (res, res), block)

    def forward(self, x):
        out = {}
        for res in self.resolution:
            res_log2 = int(np.log2(res))
            x = getattr(self, 'EncConv_Block_%dx%d' % (res, res))(x)
            out[res_log2] = x

        return out

In [ ]:
class ToStyle(nn.Module):
    def __init__(self, in_channels, out_channels, activation, drop_rate):
        super().__init__()
        self.conv = nn.Sequential(
                Conv2dLayer(in_channels=in_channels, out_channels=in_channels, kernel_size=3, activation=activation, down=2),
                Conv2dLayer(in_channels=in_channels, out_channels=in_channels, kernel_size=3, activation=activation, down=2),
                Conv2dLayer(in_channels=in_channels, out_channels=in_channels, kernel_size=3, activation=activation, down=2),
                )

        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = FullyConnectedLayer(in_features=in_channels,
                                      out_features=out_channels,
                                      activation=activation)
        # self.dropout = nn.Dropout(drop_rate)

    def forward(self, x):
        x = self.conv(x)
        x = self.pool(x)
        x = self.fc(x.flatten(start_dim=1))
        # x = self.dropout(x)

        return x

In [ ]:
class DecBlockFirstV2(nn.Module):
    def __init__(self, res, in_channels, out_channels, activation, style_dim, use_noise, demodulate, img_channels):
        super().__init__()
        self.res = res

        self.conv0 = Conv2dLayer(in_channels=in_channels,
                                out_channels=in_channels,
                                kernel_size=3,
                                activation=activation,
                                )
        self.conv1 = StyleConv(in_channels=in_channels,
                              out_channels=out_channels,
                              style_dim=style_dim,
                              resolution=2**res,
                              kernel_size=3,
                              use_noise=use_noise,
                              activation=activation,
                              demodulate=demodulate,
                              )
        self.toRGB = ToRGB(in_channels=out_channels,
                           out_channels=img_channels,
                           style_dim=style_dim,
                           kernel_size=1,
                           demodulate=False,
                           )

    def forward(self, x, ws, gs, E_features, noise_mode='random'):
        # x = self.fc(x).view(x.shape[0], -1, 4, 4)
        x = self.conv0(x)
        x = x + E_features[self.res]
        style = get_style_code(ws[:, 0], gs)
        x = self.conv1(x, style, noise_mode=noise_mode)
        style = get_style_code(ws[:, 1], gs)
        img = self.toRGB(x, style, skip=None)

        return x, img


In [ ]:
class DecBlock(nn.Module):
    def __init__(self, res, in_channels, out_channels, activation, style_dim, use_noise, demodulate, img_channels):  # res = 4, ..., resolution_log2
        super().__init__()
        self.res = res

        self.conv0 = StyleConv(in_channels=in_channels,
                               out_channels=out_channels,
                               style_dim=style_dim,
                               resolution=2**res,
                               kernel_size=3,
                               up=2,
                               use_noise=use_noise,
                               activation=activation,
                               demodulate=demodulate,
                               )
        self.conv1 = StyleConv(in_channels=out_channels,
                               out_channels=out_channels,
                               style_dim=style_dim,
                               resolution=2**res,
                               kernel_size=3,
                               use_noise=use_noise,
                               activation=activation,
                               demodulate=demodulate,
                               )
        self.toRGB = ToRGB(in_channels=out_channels,
                           out_channels=img_channels,
                           style_dim=style_dim,
                           kernel_size=1,
                           demodulate=False,
                           )

    def forward(self, x, img, ws, gs, E_features, noise_mode='random'):
        style = get_style_code(ws[:, self.res * 2 - 9], gs)
        x = self.conv0(x, style, noise_mode=noise_mode)
        x = x + E_features[self.res]
        style = get_style_code(ws[:, self.res * 2 - 8], gs)
        x = self.conv1(x, style, noise_mode=noise_mode)
        style = get_style_code(ws[:, self.res * 2 - 7], gs)
        img = self.toRGB(x, style, skip=img)

        return x, img

In [ ]:
class Decoder(nn.Module):
    def __init__(self, res_log2, activation, style_dim, use_noise, demodulate, img_channels):
        super().__init__()
        self.Dec_16x16 = DecBlockFirstV2(4, nf(4), nf(4), activation, style_dim, use_noise, demodulate, img_channels)
        for res in range(5, res_log2 + 1):
            setattr(self, 'Dec_%dx%d' % (2 ** res, 2 ** res),
                    DecBlock(res, nf(res - 1), nf(res), activation, style_dim, use_noise, demodulate, img_channels))
        self.res_log2 = res_log2

    def forward(self, x, ws, gs, E_features, noise_mode='random'):
        x, img = self.Dec_16x16(x, ws, gs, E_features, noise_mode=noise_mode)
        for res in range(5, self.res_log2 + 1):
            block = getattr(self, 'Dec_%dx%d' % (2 ** res, 2 ** res))
            x, img = block(x, img, ws, gs, E_features, noise_mode=noise_mode)

        return img



In [ ]:
class DecStyleBlock(nn.Module):
    def __init__(self, res, in_channels, out_channels, activation, style_dim, use_noise, demodulate, img_channels):
        super().__init__()
        self.res = res

        self.conv0 = StyleConv(in_channels=in_channels,
                               out_channels=out_channels,
                               style_dim=style_dim,
                               resolution=2**res,
                               kernel_size=3,
                               up=2,
                               use_noise=use_noise,
                               activation=activation,
                               demodulate=demodulate,
                               )
        self.conv1 = StyleConv(in_channels=out_channels,
                               out_channels=out_channels,
                               style_dim=style_dim,
                               resolution=2**res,
                               kernel_size=3,
                               use_noise=use_noise,
                               activation=activation,
                               demodulate=demodulate,
                               )
        self.toRGB = ToRGB(in_channels=out_channels,
                           out_channels=img_channels,
                           style_dim=style_dim,
                           kernel_size=1,
                           demodulate=False,
                           )

    def forward(self, x, img, style, skip, noise_mode='random'):
        x = self.conv0(x, style, noise_mode=noise_mode)
        x = x + skip
        x = self.conv1(x, style, noise_mode=noise_mode)
        img = self.toRGB(x, style, skip=img)

        return x, img


In [ ]:
class FirstStage(nn.Module):
    def __init__(self, img_channels, img_resolution=256, dim=180, w_dim=512, use_noise=False, demodulate=True, activation='lrelu'):
        super().__init__()
        res = 64

        self.conv_first = Conv2dLayerPartial(in_channels=img_channels+1, out_channels=dim, kernel_size=3, activation=activation)
        self.enc_conv = nn.ModuleList()
        down_time = int(np.log2(img_resolution // res))
        for i in range(down_time):  # from input size to 64
            self.enc_conv.append(
                Conv2dLayerPartial(in_channels=dim, out_channels=dim, kernel_size=3, down=2, activation=activation)
            )

        # from 64 -> 16 -> 64
        depths = [2, 3, 4, 3, 2]
        ratios = [1, 1/2, 1/2, 2, 2]
        num_heads = 6
        window_sizes = [8, 16, 16, 16, 8]
        drop_path_rate = 0.1
        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, sum(depths))]

        self.tran = nn.ModuleList()
        for i, depth in enumerate(depths):
            res = int(res * ratios[i])
            if ratios[i] < 1:
                merge = PatchMerging(dim, dim, down=int(1/ratios[i]))
            elif ratios[i] > 1:
                merge = PatchUpsampling(dim, dim, up=ratios[i])
            else:
                merge = None
            self.tran.append(
                BasicLayer(dim=dim, input_resolution=[res, res], depth=depth, num_heads=num_heads,
                           window_size=window_sizes[i], drop_path=dpr[sum(depths[:i]):sum(depths[:i + 1])],
                           downsample=merge)
            )

        # global style
        down_conv = []
        for i in range(int(np.log2(16))):
            down_conv.append(Conv2dLayer(in_channels=dim, out_channels=dim, kernel_size=3, down=2, activation=activation))
        down_conv.append(nn.AdaptiveAvgPool2d((1, 1)))
        self.down_conv = nn.Sequential(*down_conv)
        self.to_style = FullyConnectedLayer(in_features=dim, out_features=dim*2, activation=activation)
        self.ws_style = FullyConnectedLayer(in_features=w_dim, out_features=dim, activation=activation)
        self.to_square = FullyConnectedLayer(in_features=dim, out_features=16*16, activation=activation)

        style_dim = dim * 3
        self.dec_conv = nn.ModuleList()
        for i in range(down_time):  # from 64 to input size
            res = res * 2
            self.dec_conv.append(DecStyleBlock(res, dim, dim, activation, style_dim, use_noise, demodulate, img_channels))

    def forward(self, images_in, masks_in, ws, noise_mode='random'):
        x = torch.cat([masks_in - 0.5, images_in * masks_in], dim=1)

        skips = []
        x, mask = self.conv_first(x, masks_in)  # input size
        skips.append(x)
        for i, block in enumerate(self.enc_conv):  # input size to 64
            x, mask = block(x, mask)
            if i != len(self.enc_conv) - 1:
                skips.append(x)

        x_size = x.size()[-2:]
        x = feature2token(x)
        mask = feature2token(mask)
        mid = len(self.tran) // 2
        for i, block in enumerate(self.tran):  # 64 to 16
            if i < mid:
                x, x_size, mask = block(x, x_size, mask)
                skips.append(x)
            elif i > mid:
                x, x_size, mask = block(x, x_size, None)
                x = x + skips[mid - i]
            else:
                x, x_size, mask = block(x, x_size, None)

                mul_map = torch.ones_like(x) * 0.5
                mul_map = F.dropout(mul_map, training=True)
                ws = self.ws_style(ws[:, -1])
                add_n = self.to_square(ws).unsqueeze(1)
                add_n = F.interpolate(add_n, size=x.size(1), mode='linear', align_corners=False).squeeze(1).unsqueeze(-1)
                x = x * mul_map + add_n * (1 - mul_map)
                gs = self.to_style(self.down_conv(token2feature(x, x_size)).flatten(start_dim=1))
                style = torch.cat([gs, ws], dim=1)

        x = token2feature(x, x_size).contiguous()
        img = None
        for i, block in enumerate(self.dec_conv):
            x, img = block(x, img, style, skips[len(self.dec_conv)-i-1], noise_mode=noise_mode)

        # ensemble
        img = img * (1 - masks_in) + images_in * masks_in

        return img


In [ ]:
class SynthesisNet(nn.Module):
    def __init__(self,
                 w_dim,                     # Intermediate latent (W) dimensionality.
                 img_resolution,            # Output image resolution.
                 img_channels   = 3,        # Number of color channels.
                 channel_base   = 32768,    # Overall multiplier for the number of channels.
                 channel_decay  = 1.0,
                 channel_max    = 512,      # Maximum number of channels in any layer.
                 activation     = 'lrelu',  # Activation function: 'relu', 'lrelu', etc.
                 drop_rate      = 0.5,
                 use_noise      = True,
                 demodulate     = True,
                 ):
        super().__init__()
        resolution_log2 = int(np.log2(img_resolution))
        assert img_resolution == 2 ** resolution_log2 and img_resolution >= 4

        self.num_layers = resolution_log2 * 2 - 3 * 2
        self.img_resolution = img_resolution
        self.resolution_log2 = resolution_log2

        # first stage
        self.first_stage = FirstStage(img_channels, img_resolution=img_resolution, w_dim=w_dim, use_noise=False, demodulate=demodulate)

        # second stage
        self.enc = Encoder(resolution_log2, img_channels, activation, patch_size=5, channels=16)
        self.to_square = FullyConnectedLayer(in_features=w_dim, out_features=16*16, activation=activation)
        self.to_style = ToStyle(in_channels=nf(4), out_channels=nf(2) * 2, activation=activation, drop_rate=drop_rate)
        style_dim = w_dim + nf(2) * 2
        self.dec = Decoder(resolution_log2, activation, style_dim, use_noise, demodulate, img_channels)

    def forward(self, images_in, masks_in, ws, noise_mode='random', return_stg1=False):
        out_stg1 = self.first_stage(images_in, masks_in, ws, noise_mode=noise_mode)

        # encoder
        x = images_in * masks_in + out_stg1 * (1 - masks_in)
        x = torch.cat([masks_in - 0.5, x, images_in * masks_in], dim=1)
        E_features = self.enc(x)

        fea_16 = E_features[4]
        mul_map = torch.ones_like(fea_16) * 0.5
        mul_map = F.dropout(mul_map, training=True)
        add_n = self.to_square(ws[:, 0]).view(-1, 16, 16).unsqueeze(1)
        add_n = F.interpolate(add_n, size=fea_16.size()[-2:], mode='bilinear', align_corners=False)
        fea_16 = fea_16 * mul_map + add_n * (1 - mul_map)
        E_features[4] = fea_16

        # style
        gs = self.to_style(fea_16)

        # decoder
        img = self.dec(fea_16, ws, gs, E_features, noise_mode=noise_mode)

        # ensemble
        img = img * (1 - masks_in) + images_in * masks_in

        if not return_stg1:
            return img
        else:
            return img, out_stg1


In [ ]:
class Generator(nn.Module):
    def __init__(self,
                 z_dim,                  # Input latent (Z) dimensionality, 0 = no latent.
                 c_dim,                  # Conditioning label (C) dimensionality, 0 = no label.
                 w_dim,                  # Intermediate latent (W) dimensionality.
                 img_resolution,         # resolution of generated image
                 img_channels,           # Number of input color channels.
                 synthesis_kwargs = {},  # Arguments for SynthesisNetwork.
                 mapping_kwargs   = {},  # Arguments for MappingNetwork.
                 ):
        super().__init__()
        self.z_dim = z_dim
        self.c_dim = c_dim
        self.w_dim = w_dim
        self.img_resolution = img_resolution
        self.img_channels = img_channels

        self.synthesis = SynthesisNet(w_dim=w_dim,
                                      img_resolution=img_resolution,
                                      img_channels=img_channels,
                                      **synthesis_kwargs)
        self.mapping = MappingNet(z_dim=z_dim,
                                  c_dim=c_dim,
                                  w_dim=w_dim,
                                  num_ws=self.synthesis.num_layers,
                                  **mapping_kwargs)

    def forward(self, images_in, masks_in, z, c, truncation_psi=1, truncation_cutoff=None, skip_w_avg_update=False,
                noise_mode='random', return_stg1=False):
        ws = self.mapping(z, c, truncation_psi=truncation_psi, truncation_cutoff=truncation_cutoff,
                          skip_w_avg_update=skip_w_avg_update)

        if not return_stg1:
            img = self.synthesis(images_in, masks_in, ws, noise_mode=noise_mode)
            return img
        else:
            img, out_stg1 = self.synthesis(images_in, masks_in, ws, noise_mode=noise_mode, return_stg1=True)
            return img, out_stg1



In [ ]:
class Discriminator(torch.nn.Module):
    def __init__(self,
                 c_dim,                        # Conditioning label (C) dimensionality.
                 img_resolution,               # Input resolution.
                 img_channels,                 # Number of input color channels.
                 channel_base       = 32768,    # Overall multiplier for the number of channels.
                 channel_max        = 512,      # Maximum number of channels in any layer.
                 channel_decay      = 1,
                 cmap_dim           = None,     # Dimensionality of mapped conditioning label, None = default.
                 activation         = 'lrelu',
                 mbstd_group_size   = 4,        # Group size for the minibatch standard deviation layer, None = entire minibatch.
                 mbstd_num_channels = 1,        # Number of features for the minibatch standard deviation layer, 0 = disable.
                 ):
        super().__init__()
        self.c_dim = c_dim
        self.img_resolution = img_resolution
        self.img_channels = img_channels

        resolution_log2 = int(np.log2(img_resolution))
        assert img_resolution == 2 ** resolution_log2 and img_resolution >= 4
        self.resolution_log2 = resolution_log2

        if cmap_dim == None:
            cmap_dim = nf(2)
        if c_dim == 0:
            cmap_dim = 0
        self.cmap_dim = cmap_dim

        if c_dim > 0:
            self.mapping = MappingNet(z_dim=0, c_dim=c_dim, w_dim=cmap_dim, num_ws=None, w_avg_beta=None)

        Dis = [DisFromRGB(img_channels+1, nf(resolution_log2), activation)]
        for res in range(resolution_log2, 2, -1):
            Dis.append(DisBlock(nf(res), nf(res-1), activation))

        if mbstd_num_channels > 0:
            Dis.append(MinibatchStdLayer(group_size=mbstd_group_size, num_channels=mbstd_num_channels))
        Dis.append(Conv2dLayer(nf(2) + mbstd_num_channels, nf(2), kernel_size=3, activation=activation))
        self.Dis = nn.Sequential(*Dis)

        self.fc0 = FullyConnectedLayer(nf(2)*4**2, nf(2), activation=activation)
        self.fc1 = FullyConnectedLayer(nf(2), 1 if cmap_dim == 0 else cmap_dim)

        # for 64x64
        Dis_stg1 = [DisFromRGB(img_channels+1, nf(resolution_log2) // 2, activation)]
        for res in range(resolution_log2, 2, -1):
            Dis_stg1.append(DisBlock(nf(res) // 2, nf(res - 1) // 2, activation))

        if mbstd_num_channels > 0:
            Dis_stg1.append(MinibatchStdLayer(group_size=mbstd_group_size, num_channels=mbstd_num_channels))
        Dis_stg1.append(Conv2dLayer(nf(2) // 2 + mbstd_num_channels, nf(2) // 2, kernel_size=3, activation=activation))
        self.Dis_stg1 = nn.Sequential(*Dis_stg1)

        self.fc0_stg1 = FullyConnectedLayer(nf(2) // 2 * 4 ** 2, nf(2) // 2, activation=activation)
        self.fc1_stg1 = FullyConnectedLayer(nf(2) // 2, 1 if cmap_dim == 0 else cmap_dim)

    def forward(self, images_in, masks_in, images_stg1, c):
        x = self.Dis(torch.cat([masks_in - 0.5, images_in], dim=1))
        x = self.fc1(self.fc0(x.flatten(start_dim=1)))

        x_stg1 = self.Dis_stg1(torch.cat([masks_in - 0.5, images_stg1], dim=1))
        x_stg1 = self.fc1_stg1(self.fc0_stg1(x_stg1.flatten(start_dim=1)))

        if self.c_dim > 0:
            cmap = self.mapping(None, c)

        if self.cmap_dim > 0:
            x = (x * cmap).sum(dim=1, keepdim=True) * (1 / np.sqrt(self.cmap_dim))
            x_stg1 = (x_stg1 * cmap).sum(dim=1, keepdim=True) * (1 / np.sqrt(self.cmap_dim))

        return x, x_stg1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pickle

# Provide the full path to your pickle file
file_path = '/content/drive/MyDrive/Places_512.pkl'

# Load the pickle file
with open(file_path, 'rb') as file:
    data = pickle.load(file)

# Use the loaded data
print(data)


{'training_set_kwargs': {'class_name': 'datasets.dataset_512_resize.ImageFolderMaskDataset', 'path': '/data/ceph/gavinqi/data/places/data_large', 'use_labels': False, 'max_size': 1803414, 'xflip': True, 'resolution': 512}, 'val_set_kwargs': {'class_name': 'datasets.dataset_512_resize.ImageFolderMaskDataset', 'path': '/data/ceph/gavinqi/data/places/validation/val_512', 'use_labels': False, 'max_size': 36500, 'xflip': False, 'resolution': 512}, 'G': Generator(
  (synthesis): SynthesisNet(
    (first_stage): FirstStage(
      (conv_first): Conv2dLayerPartial(
        (conv): Conv2dLayer()
      )
      (enc_conv): ModuleList(
        (0-2): 3 x Conv2dLayerPartial(
          (conv): Conv2dLayer()
        )
      )
      (tran): ModuleList(
        (0): BasicLayer(
          (blocks): ModuleList(
            (0-1): 2 x SwinTransformerBlock(
              (attn): WindowAttention(
                (q): FullyConnectedLayer()
                (k): FullyConnectedLayer()
                (v): FullyC

In [ ]:
import cv2
import numpy as np
import PIL.Image
import torch
import torch.nn.functional as F

import legacy


def load_network(network_pkl, device):
    """Load the pretrained network."""
    print(f'Loading network from: {network_pkl}')
    with open(network_pkl, 'rb') as f:
        G_saved = legacy.load_network_pkl(f)['G_ema'].to(device).eval().requires_grad_(False) # type: ignore
    return G_saved


def read_image(image_path):
    """Read an image file into a NumPy array."""
    with open(image_path, 'rb') as f:
        image = np.array(PIL.Image.open(f))
    if image.ndim == 2:
        image = image[:, :, np.newaxis]  # HW => HWC
        image = np.repeat(image, 3, axis=2)
    image = image.transpose(2, 0, 1)  # HWC => CHW
    image = image[:3]  # Keep only 3 channels (RGB)
    return image


import torch
import cv2
import numpy as np
import os
from PIL import Image
from torchvision.transforms.functional import to_tensor, to_pil_image




from PIL import Image
import numpy as np
import torch
import os
import cv2

def process_single_image(network_pkl, image_path, mask_path, outdir, resolution=512, truncation_psi=1.0, noise_mode='const'):
    """
    Process a single image using the pretrained network and a provided mask.
    Resize both the image and mask to 512x512 before processing.
    """
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Load network
    G_saved = load_network(network_pkl, device)

    # Create a new Generator instance with matching resolution
    G = Generator(z_dim=512, c_dim=0, w_dim=512, img_resolution=resolution, img_channels=3).to(device).eval().requires_grad_(False)

    # Copy parameters
    for src_param, dest_param in zip(G_saved.parameters(), G.parameters()):
        dest_param.data.copy_(src_param.data)

    # Prepare input image
    # Resize image to 512x512
    input_image = Image.open(image_path).convert('RGB')
    input_image = input_image.resize((256, 256), Image.Resampling.LANCZOS)
    image = np.array(input_image).astype(np.float32)
    image = (torch.from_numpy(image).float().to(device) / 127.5 - 1).permute(2, 0, 1).unsqueeze(0)  # [1, 3, 512, 512]

    # Load and process the provided mask
    # Resize mask to 512x512
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE).astype(np.float32) / 255.0
    mask = cv2.resize(mask, (256, 256), interpolation=cv2.INTER_NEAREST)  # Ensure exact 512x512
    mask = torch.from_numpy(mask).float().to(device).unsqueeze(0).unsqueeze(0)  # [1, 1, 512, 512]

    # Debugging: Check tensor shapes
    print(f"Image shape: {image.shape}")  # Expecting [1, 3, 512, 512]
    print(f"Mask shape: {mask.shape}")  # Expecting [1, 1, 512, 512]

    # Generate a random latent vector
    z = torch.from_numpy(np.random.randn(1, G.z_dim)).to(device)
    label = torch.zeros([1, G.c_dim], device=device)  # No labels

    # Generate output
    with torch.no_grad():
        try:
            output = G(image, mask, z, label, truncation_psi=truncation_psi, noise_mode=noise_mode)
        except RuntimeError as e:
            print(f"Error during generation: {e}")
            return

        output = (output.permute(0, 2, 3, 1) * 127.5 + 127.5).round().clamp(0, 255).to(torch.uint8)
        output = output[0].cpu().numpy()

    # Save output image
    output_image = Image.fromarray(output, 'RGB')
    output_path = os.path.join(outdir, 'output_image.png')
    output_image.save(output_path)
    print(f"Output saved at: {output_path}")

# Run the function
# Paths
network_pkl_path = '/content/drive/MyDrive/Places_512.pkl'  # Path to your pretrained .pkl file
input_image_path = '/content/Places365_test_00000001.jpg'  # Path to your input image
mask_image_path = '/content/000003.png'  # Path to your mask image
output_directory = '/content/output'  # Directory to save the output image

# Ensure the output directory exists
import os
os.makedirs(output_directory, exist_ok=True)

# Run the function
process_single_image(network_pkl_path, input_image_path, mask_image_path, output_directory)


Loading network from: /content/drive/MyDrive/Places_512.pkl
Image shape: torch.Size([1, 3, 256, 256])
Mask shape: torch.Size([1, 1, 256, 256])
Error during generation: The size of tensor a (256) must match the size of tensor b (128) at non-singleton dimension 3


In [ ]:
!git clone https://github.com/fenglinglwb/MAT.git
%cd MAT

fatal: destination path 'MAT' already exists and is not an empty directory.
/content/MAT


In [ ]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.5/192.5 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 25.3 MB/s eta 0:00:00
  Attempting uninstall: imageio-ffmpeg
    Found existing installation: imageio-ffmpeg 0.5.1
    Uninstalling imageio-ffmpeg-0.5.1:
      Successfully uninstalled imageio-ffmpeg-0.5.1


In [ ]:
!python generate_image.py --network /content/drive/MyDrive/Places_512.pkl --dpath /content/data --mpath /content/Un --outdir /content/output

/usr/local/lib/python3.10/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
Loading data from: /content/data
Loading mask from: /content/Un
Loading networks from: /content/drive/MyDrive/Places_512.pkl
Prcessing: 31.png
Setting up PyTorch plugin "bias_act_plugin"... /usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... /usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please se

In [ ]:
import os
from PIL import Image

def resize_images_in_folder(folder_path, target_size=(512, 512)):
    """
    Resize all images in the given folder to the target size and replace the originals.

    Parameters:
    folder_path (str): Path to the folder containing images.
    target_size (tuple): The target size for resizing (width, height).
    """
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        # Check if it's a valid image file
        try:
            with Image.open(file_path) as img:
                # Resize the image
                img_resized = img.resize(target_size)

                # Save the resized image in the same path
                img_resized.save(file_path)

                print(f"Resized and replaced: {filename}")
        except Exception as e:
            print(f"Skipping file {filename}: {e}")

# Example usage:
folder_path = "/content/data"  # Replace with your folder path
resize_images_in_folder(folder_path)



Resized and replaced: 1.jpg


In [ ]:
from PIL import Image
import numpy as np

def apply_mask(input_image_path, mask_image_path, output_image_path):
    # Open the input image and the mask image
    input_image = Image.open(input_image_path).convert("RGBA")  # Convert to RGBA to support transparency
    mask_image = Image.open(mask_image_path).convert("L")  # Convert mask to grayscale (L mode)

    # Ensure the mask is the same size as the input image
    mask_image = mask_image.resize(input_image.size)

    # Create a new image where the mask is applied
    input_image_np = np.array(input_image)
    mask_image_np = np.array(mask_image)

    # Apply the mask: we will keep the image where the mask is white and make the masked area transparent
    # If the mask pixel is 0 (black), the output pixel will be transparent (0,0,0,0)
    # If the mask pixel is 255 (white), the output pixel will retain the input image pixel

    # Create an output image with transparency for masked areas
    output_image_np = np.zeros_like(input_image_np)

    # Where the mask is white, retain the input image pixel values
    output_image_np[mask_image_np == 255] = input_image_np[mask_image_np == 255]

    # Convert the numpy array back to an image
    output_image = Image.fromarray(output_image_np, "RGBA")

    # Save the output image
    output_image.save(output_image_path)

    # Optional: Show the output image
    output_image.show()

# Example usage:
apply_mask('/content/drive/MyDrive/Data/9.jpg', '/content/drive/MyDrive/Masks/000020.png', '9.png')
